# Import Libraries

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json
import ast
from rapidfuzz import fuzz
from datetime import datetime
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document
import time

c:\Codes\Python\notebooks\echo_chatbot\.chatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Initialization

In [3]:
# Get API Keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
# index = pc.Index("echo-openai")

# OpenAI Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)

C:\Users\Zeck Lowe\AppData\Local\Temp\ipykernel_26068\1551298541.py:10: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(api_key=OPENAI_API_KEY)


# Chatbot

In [7]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

docsearch = PineconeVectorStore(
    index_name=input("Give organization name"),
    embedding=EMBEDDINGS,
    namespace=input("Give meeting title")
)

retriever = docsearch.as_retriever()

LLM = ChatOpenAI(temperature=0.0, model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

combine_docs_chain = create_stuff_documents_chain(
    LLM, retrieval_qa_chat_prompt
)

retrieval_chain = create_retrieval_chain(
    retriever, combine_docs_chain
)

c:\Codes\Python\notebooks\echo_chatbot\.chatbot\Lib\site-packages\langsmith\client.py:351: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# Query

In [8]:
query1 = "What was the QA perspective on the kickoff meeting?"

query2 = "What are the key points of the meeting?"

### Query without relevant knowledge

In [9]:
answer1_without_knowledge = LLM.invoke(query1)

print("Query 1:", query1)
print("\nAnswer without knowledge:\n\n", answer1_without_knowledge.content)
print("\n")
time.sleep(2)

Query 1: What was the QA perspective on the kickoff meeting?

Answer without knowledge:

 I don't have specific details about a particular kickoff meeting as my responses are generated based on a mixture of licensed data, data created by human trainers, and publicly available information. I can provide a general overview of what a QA (Quality Assurance) perspective might involve in a kickoff meeting:

1. **Understanding Requirements**: QA professionals would focus on understanding the project requirements clearly. They would ensure that they have a comprehensive understanding of what needs to be tested and the project deliverables.

2. **Clarifying Objectives**: They would seek to clarify the testing objectives and ensure these are aligned with the overall project goals.

3. **Resource Allocation**: From a QA perspective, it's crucial to discuss and secure the necessary resources for testing. This includes personnel, tools, and time.

4. **Timeline and Milestones**: QA would be interes

### Query with Knowledge

In [10]:
answer1_with_knowledge = retrieval_chain.invoke({"input": query1})

print("Query 1:", query1)
print("\nAnswer with knowledge:\n\n", answer1_with_knowledge['answer'])
print("\nContext used:\n\n", answer1_with_knowledge['context'])
print("\n")
time.sleep(2)

Query 1: What was the QA perspective on the kickoff meeting?

Answer with knowledge:

 Sara, the QA analyst, expressed her role in the project during the kickoff meeting. She mentioned that she would be responsible for testing the software to ensure it meets quality standards and is free of bugs. This indicates her focus on quality assurance and her commitment to delivering a reliable product.

Context used:

 [Document(id='5285811d-1239-4477-822d-805724435aad', metadata={}, page_content="[00:00:00] John: Good morning, everyone. Thank you for joining today's kickoff meeting for our new     software development project. We'll be discussing the project scope, timelines, and\nresponsibilities. Let's get started with a quick round of introductions. I'll go first. I'm John, the     project manager. I'll be overseeing the project and ensuring we stay on track. Alice, would you     like to go next?\n[00:00:20]\nAlice: Sure, thanks John. Hi, everyone. I'm Alice, the lead developer. I'll be res